<a href="https://colab.research.google.com/github/halfspin-qc/aiapps/blob/main/chatbot_opensource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Mar 27 21:51:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install -q langchain gpt-index llama-index transformers sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.3/426.3 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTSimpleVectorIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor
import torch
from langchain.llms.base import LLM
from transformers import pipeline


In [4]:
class FlanLLM(LLM):
    model_name = "google/flan-t5-base"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
 
    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=FlanLLM())


In [5]:

hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)


In [6]:
text1 = """
Albert Einstein ( 14 March 1879 – 18 April 1955) was a German-born theoretical physicist,[5] widely acknowledged to be one of the greatest and most influential physicists of all time.[citation needed] Einstein is best known for developing the theory of relativity, but he also made important contributions to the development of the theory of quantum mechanics. Relativity and quantum mechanics are the two pillars of modern physics.[1][6] His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been dubbed "the world's most famous equation".[7] His work is also known for its influence on the philosophy of science.[8][9] He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect",[10] a pivotal step in the development of quantum theory. His intellectual achievements and originality resulted in "Einstein" becoming synonymous with "genius".[11] Einsteinium, one of the synthetic elements in the periodic table, was named in his honor.[12]

In 1905, a year sometimes described as his annus mirabilis ('miracle year'), Einstein published four groundbreaking papers.[13] These outlined the theory of the photoelectric effect, explained Brownian motion, introduced special relativity, and demonstrated mass–energy equivalence. Einstein thought that the laws of classical mechanics could no longer be reconciled with those of the electromagnetic field, which led him to develop his special theory of relativity. He then extended the theory to gravitational fields; he published a paper on general relativity in 1916, introducing his theory of gravitation. In 1917, he applied the general theory of relativity to model the structure of the universe.[14][15] He continued to deal with problems of statistical mechanics and quantum theory, which led to his explanations of particle theory and the motion of molecules. He also investigated the thermal properties of light and the quantum theory of radiation, which laid the foundation of the photon theory of light.

However, for much of the later part of his career, he worked on two ultimately unsuccessful endeavors. First, despite his great contributions to quantum mechanics, he opposed what it evolved into, objecting that "God does not play dice".[16] Second, he attempted to devise a unified field theory by generalizing his geometric theory of gravitation to include electromagnetism. As a result, he became increasingly isolated from the mainstream of modern physics.

Einstein was born in the German Empire, but moved to Switzerland in 1895, forsaking his German citizenship (as a subject of the Kingdom of Württemberg)[note 1] the following year. In 1897, at the age of 17, he enrolled in the mathematics and physics teaching diploma program at the Swiss Federal polytechnic school in Zürich, graduating in 1900. In 1901, he acquired Swiss citizenship, which he kept for the rest of his life, and in 1903 he secured a permanent position at the Swiss Patent Office in Bern. In 1905, he was awarded a PhD by the University of Zurich. In 1914, Einstein moved to Berlin in order to join the Prussian Academy of Sciences and the Humboldt University of Berlin. In 1917, Einstein became director of the Kaiser Wilhelm Institute for Physics; he also became a German citizen again, this time Prussian.

In 1933, while Einstein was visiting the United States, Adolf Hitler came to power in Germany. Einstein, as a Jew, objected to the policies of the newly elected Nazi government;[17] he settled in the United States and became an American citizen in 1940.[18] On the eve of World War II, he endorsed a letter to President Franklin D. Roosevelt alerting him to the potential German nuclear weapons program and recommending that the US begin similar research. Einstein supported the Allies but generally denounced the idea of nuclear weapons.[19]

Life and career
Early life and education
See also: Einstein family
A young boy with short hair and a round face, wearing a white collar and large bow, with vest, coat, skirt, and high boots. He is leaning against an ornate chair.
Einstein at the age of three in 1882
Studio photo of a boy seated in a relaxed posture and wearing a suit, posed in front of a backdrop of scenery.
Einstein in 1893 (age 14)
Albert Einstein was born in Ulm,[5] in the Kingdom of Württemberg in the German Empire, on 14 March 1879 into a family of secular Ashkenazi Jews.[20][21] His parents were Hermann Einstein, a salesman and engineer, and Pauline Koch. In 1880, the family moved to Munich, where Einstein's father and his uncle Jakob founded Elektrotechnische Fabrik J. Einstein & Cie, a company that manufactured electrical equipment based on direct current.[5]

Albert attended a Catholic elementary school in Munich, from the age of five, for three years. At the age of eight, he was transferred to the Luitpold-Gymnasium (now known as the Albert-Einstein-Gymnasium), where he received advanced primary and secondary school education until he left the German Empire seven years later.[22]

In 1894, Hermann and Jakob's company lost a bid to supply the city of Munich with electrical lighting because they lacked the capital to convert their equipment from the direct current (DC) standard to the more efficient alternating current (AC) standard.[23] The loss forced the sale of the Munich factory. In search of business, the Einstein family moved to Italy, first to Milan and a few months later to Pavia. In Pavia, the Einsteins settled in Palazzo Cornazzani, a medieval building where, at different times, Ugo Foscolo, Contardo Ferrini and Ada Negri lived.[24] When the family moved to Pavia, Einstein, then 15, stayed in Munich to finish his studies at the Luitpold Gymnasium. His father intended for him to pursue electrical engineering, but Einstein clashed with the authorities and resented the school's regimen and teaching method. He later wrote that the spirit of learning and creative thought was lost in strict rote learning. At the end of December 1894, he traveled to Italy to join his family in Pavia, convincing the school to let him go by using a doctor's note.[25] During his time in Italy, he wrote a short essay with the title "On the Investigation of the State of the Ether in a Magnetic Field".[26][27]

Einstein excelled at math and physics from a young age, reaching a mathematical level years ahead of his peers. The 12-year-old Einstein taught himself algebra and Euclidean geometry over a single summer.[28] Einstein also independently discovered his own original proof of the Pythagorean theorem aged 12.[29] A family tutor Max Talmud says that after he had given the 12-year-old Einstein a geometry textbook, after a short time "[Einstein] had worked through the whole book. He thereupon devoted himself to higher mathematics ... Soon the flight of his mathematical genius was so high I could not follow."[30] His passion for geometry and algebra led the 12-year-old to become convinced that nature could be understood as a "mathematical structure".[30] Einstein started teaching himself calculus at 12, and as a 14-year-old he says he had "mastered integral and differential calculus".[31]

At the age of 13, when he had become more seriously interested in philosophy (and music),[32] Einstein was introduced to Kant's Critique of Pure Reason. Kant became his favorite philosopher, his tutor stating: "At the time he was still a child, only thirteen years old, yet Kant's works, incomprehensible to ordinary mortals, seemed to be clear to him."[30]

Einstein's matriculation certificate at the age of 17. The heading translates as "The Education Committee of the Canton of Aargau". His scores were German 5, French 3, Italian 5, History 6, Geography 4, Algebra 6, Geometry 6, Descriptive Geometry 6, Physics 6, Chemistry 5, Natural History 5, Art Drawing 4, Technical Drawing 4. 6 = very good, 5 = good, 4 = sufficient, 3 = insufficient, 2 = poor, 1 = very poor.
Einstein's Matura certificate, 1896[note 2]
In 1895, at the age of 16, Einstein took the entrance examinations for the Swiss Federal polytechnic school in Zürich (later the Eidgenössische Technische Hochschule, ETH). He failed to reach the required standard in the general part of the examination,[33] but obtained exceptional grades in physics and mathematics.[34] On the advice of the principal of the polytechnic school, he attended the Argovian cantonal school (gymnasium) in Aarau, Switzerland, in 1895 and 1896 to complete his secondary schooling. While lodging with the family of Jost Winteler, he fell in love with Winteler's daughter, Marie. Einstein's sister Maja later married Winteler's son Paul.[35] In January 1896, with his father's approval, Einstein renounced his citizenship in the German Kingdom of Württemberg to avoid military service.[36] In September 1896 he passed the Swiss Matura with mostly good grades, including a top grade of 6 in physics and mathematical subjects, on a scale of 1–6.[37] At 17, he enrolled in the four-year mathematics and physics teaching diploma program at the Federal polytechnic school. Marie Winteler, who was a year older, moved to Olsberg, Switzerland, for a teaching post.[35]

Einstein's future wife, a 20-year-old Serbian named Mileva Marić, also enrolled at the polytechnic school that year. She was the only woman among the six students in the mathematics and physics section of the teaching diploma course. Over the next few years, Einstein's and Marić's friendship developed into a romance, and they spent countless hours debating and reading books together on extra-curricular physics in which they were both interested. Einstein wrote in his letters to Marić that he preferred studying alongside her.[38] In 1900, Einstein passed the exams in Maths and Physics and was awarded a Federal teaching diploma.[39] There is eyewitness evidence and several letters over many years that indicate Marić might have collaborated with Einstein prior to his landmark 1905 papers,[38][40][41] known as the Annus Mirabilis papers, and that they developed some of the concepts together during their studies, although some historians of physics who have studied the issue disagree that she made any substantive contributions.[42][43][44][45]

Marriages and children

Albert Einstein and Mileva Marić Einstein, 1912
Early correspondence between Einstein and Marić was discovered and published in 1987 which revealed that the couple had a daughter named "Lieserl", born in early 1902 in Novi Sad where Marić was staying with her parents. Marić returned to Switzerland without the child, whose real name and fate are unknown. The contents of Einstein's letter in September 1903 suggest that the girl was either given up for adoption or died of scarlet fever in infancy.[46][47]

Einstein and Marić married in January 1903. In May 1904, their son Hans Albert Einstein was born in Bern, Switzerland. Their son Eduard was born in Zürich in July 1910. The couple moved to Berlin in April 1914, but Marić returned to Zürich with their sons after learning that, despite their close relationship before,[38] Einstein's chief romantic attraction was now his cousin Elsa Löwenthal;[48] she was his first cousin maternally and second cousin paternally.[49] Einstein and Marić divorced on 14 February 1919, having lived apart for five years.[50][51] As part of the divorce settlement, Einstein agreed to give Marić any future (in the event, 1921) Nobel Prize money.[52]

In letters revealed in 2015, Einstein wrote to his early love Marie Winteler about his marriage and his strong feelings for her. He wrote in 1910, while his wife was pregnant with their second child: "I think of you in heartfelt love every spare minute and am so unhappy as only a man can be." He spoke about a "misguided love" and a "missed life" regarding his love for Marie.[53]

Einstein married Löwenthal in 1919,[54][55] after having had a relationship with her since 1912.[49][56] They emigrated to the United States in 1933. Elsa was diagnosed with heart and kidney problems in 1935 and died in December 1936.[57]

In 1923, Einstein fell in love with a secretary named Betty Neumann, the niece of a close friend, Hans Mühsam.[58][59][60][61] In a volume of letters released by Hebrew University of Jerusalem in 2006,[62] Einstein described about six women, including Margarete Lebach (a blonde Austrian), Estella Katzenellenbogen (the rich owner of a florist business), Toni Mendel (a wealthy Jewish widow) and Ethel Michanowski (a Berlin socialite), with whom he spent time and from whom he received gifts while being married to Elsa.[63][64] Later, after the death of his second wife Elsa, Einstein was briefly in a relationship with Margarita Konenkova. Konenkova was a Russian spy who was married to the Russian sculptor Sergei Konenkov (who created the bronze bust of Einstein at the Institute for Advanced Study at Princeton).[65][66][failed verification]

Einstein's son Eduard had a breakdown at about age 20 and was diagnosed with schizophrenia.[67] His mother cared for him and he was also committed to asylums for several periods, finally, after her death, being committed permanently to Burghölzli, the Psychiatric University Hospital in Zürich.[68]

Patent office
Head and shoulders shot of a young, moustached man with dark, curly hair wearing a plaid suit and vest, striped shirt, and a dark tie.
Einstein in 1904 (age 25)
After graduating in 1900, Einstein spent almost two years searching for a teaching post. He acquired Swiss citizenship in February 1901,[69] but was not conscripted for medical reasons. With the help of Marcel Grossmann's father, he secured a job in Bern at the Swiss Patent Office,[70][71] as an assistant examiner – level III.[72][73]

Einstein evaluated patent applications for a variety of devices including a gravel sorter and an electromechanical typewriter.[73] In 1903, his position at the Swiss Patent Office became permanent, although he was passed over for promotion until he "fully mastered machine technology".[74]

Much of his work at the patent office related to questions about transmission of electric signals and electrical-mechanical synchronization of time, two technical problems that show up conspicuously in the thought experiments that eventually led Einstein to his radical conclusions about the nature of light and the fundamental connection between space and time.[13]

With a few friends he had met in Bern, Einstein started a small discussion group in 1902, self-mockingly named "The Olympia Academy", which met regularly to discuss science and philosophy. Sometimes they were joined by Mileva who attentively listened but did not participate.[75] Their readings included the works of Henri Poincaré, Ernst Mach, and David Hume, which influenced his scientific and philosophical outlook.[76]
"""
     

In [7]:

#documents = SimpleDirectoryReader('data').load_data()

from llama_index import Document

text_list = [text1]

documents = [Document(t) for t in text_list]

In [8]:

# # set number of output tokens
num_output = 500
# # set maximum input size
max_input_size = 512
# # set maximum chunk overlap
max_chunk_overlap = 15


prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [9]:


index = GPTSimpleVectorIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

# index = GPTListIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor, prompt_helper =prompt_helper)

#index.save_to_disk('index.json')



In [10]:

import logging

logging.getLogger().setLevel(logging.CRITICAL)


In [12]:
response = index.query( "What was the title of the short essay Einstein wrote in Italy?") 
print(response)

On the Investigation of the State of the Ether in a Magnetic Field


In [20]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [22]:
! pip install -q gradio

In [23]:
import gradio as gr

In [25]:
index = None

In [26]:
# Get input text
# Build the llama Index
# Return GPT simple vector index


def build_the_bot(input_text):
  text_list = [input_text]
  documents = [Document(t) for t in text_list]
  global index
  index = GPTSimpleVectorIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)
  return('Index built successfully')

In [27]:
# Function to create a chat interface with history
# Query the index with the user Input, build a response and add it to the chat history


def chat(chat_history, user_input):
  
  bot_response = index.query(user_input)
  #print(bot_response)
  response = ""
  for letter in ''.join(bot_response.response): #[bot_response[i:i+1] for i in range(0, len(bot_response), 1)]:
      response += letter + ""
      yield chat_history + [(user_input, response)]

In [28]:
# UI with Gradio


with gr.Blocks() as demo:
    gr.Markdown('# New Chatbot!')
    with gr.Tab("Input Text Here"):
        text_input = gr.Textbox()
        text_output = gr.Textbox()
        text_button = gr.Button("Build the Bot!!!")
        text_button.click(build_the_bot, text_input, text_output)
    with gr.Tab("Google Flan-Base!"):
#          inputbox = gr.Textbox("Input your text to build a Q&A Bot here.....")
          chatbot = gr.Chatbot()
          message = gr.Textbox ("What is this document about?")
          message.submit(chat, [chatbot, message], chatbot)

demo.queue().launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ea9f39e58a4423609f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ea9f39e58a4423609f.gradio.live
